<a href="https://colab.research.google.com/github/yaswanth1701/mnist_classifer/blob/main/mnist_digit_classifer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 

In [2]:
from keras.datasets import mnist

In [3]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()

In [4]:
print(train_X.shape[0])

60000


In [5]:
print(train_X[0].shape)

(28, 28)


In [6]:
class DNN:
  def __init__(self,size,epochs,lr):
    self.size=size
    self.epochs=epochs
    self.lr=lr


    input_layer=size[0]
    hidden1_layer=size[1]
    hidden2_layer=size[2]
    output_layer=size[3]
    
    self.params={ 
        'W1':np.random.randn(hidden1_layer,input_layer)*np.sqrt(1./hidden1_layer),
        'W2':np.random.randn(hidden2_layer,hidden1_layer)*np.sqrt(1./hidden2_layer),
        'W3':np.random.randn(output_layer,hidden2_layer)*np.sqrt(1./output_layer)
    }
  def sigmoid(self,x,derivate=False):
    if derivate:
      return (np.exp(-x))/((np.exp(-x)+1)**2)
    return 1/(1+np.exp(-x))
  def softmax(self,x,derivate=False):
    exps= np.exp(x-x.max())
    if derivate:
      return exps/np.sum(exps,axis=0)*(1-exps/np.sum(exps,axis=0))
    return exps / np.sum(exps, axis=0)


  def forward_pass(self,x_train):
    params=self.params
    params['A0']=x_train # (28*28,1)
    # input layers to hidden1_layer
    params['Z1']=np.dot(params['W1'],params['A0']) # (128,784)*(784,1)=(128x1)
    params['A1']=self.sigmoid(params['Z1']) # 128x1 sigmoid

    #input to hidden2_layer
    params['Z2']=np.dot(params['W2'],params['A1']) #(64,125)*(125,1)=(64x1)
    params['A2']=self.sigmoid(params['Z2']) # 64x1 sigmoid
    #input to output_layer
    params['Z3']=np.dot(params['W3'],params['A2']) # (10,64)*(64,1)=(10x1)
    params['A3']= self.softmax(params['Z3'])         # 10x1    softmax 
    return params['A3']


  def backward_pass(self,output,y_train):
    params=self.params
    change_w={}
    # calculate w3 update
    y_train=y_train.reshape(10,1)
    error=2*(output-y_train)/output.shape[0]*self.softmax(params['Z3'],derivate=True)
    change_w['W3']=np.outer(error,params['A2'])

    # calculate w2 update
    error=np.dot(params['W3'].T,error)*self.sigmoid(params['Z2'],derivate=True)
    change_w['W2']=np.outer(error,params['A1'])
    # calculate w1 update
    error=np.dot(params['W2'].T,error)*self.sigmoid(params['Z1'],derivate=True)
    change_w['W1']= np.outer(error,params['A0'])
    return change_w

  def update_weights(self,change_w):
    for key,val in change_w.items():
      self.params[key]-=val*self.lr


  def compute_accuacry(self,test_X,test_Y):
    prediction=np.zeros(10000)
    for i in range(10000):
      input=(test_X[i].reshape(784,1)/255*0.99)+0.01
      target=np.zeros(10)+0.01
      target[test_Y[i]]=0.99
      output=self.forward_pass(input)
      pred=np.argmax(output)
      prediction[i]=(pred==np.argmax(target))
    return np.mean(prediction)



  def train(self,train_X,train_Y,test_X,test_Y ):
    for i in range(self.epochs):
      for j in range(60000):
        input=(train_X[j].reshape(784,1)/255*0.99)+0.01
        target=np.zeros(10)+0.01
        target[train_Y[j]]=0.99
        output=self.forward_pass(input)
        change_w=self.backward_pass(output,target)
        self.update_weights(change_w)
      accuracy=self.compute_accuacry(test_X,test_Y)
      print('epochs{epoch} --- accuaracy{a} %'.format(epoch=i,a=accuracy*100))


In [8]:
dnn=DNN([784,128,64,10],10,0.05)
dnn.train(train_X, train_y,test_X, test_y)


KeyboardInterrupt: ignored

In [26]:
predict=dnn.forward_pass((test_X[9100].reshape(784,1)/255*0.99)+0.01)
print(np.argmax(predict),test_y[9100])

3 3
